In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [2]:
# Memory Stores
# We'll create stores for both short-term (chat history) and long-term memory.

chat_store = {}
long_term_memory = {}

def get_chat_history(session_id: str):
    if session_id not in chat_store:
        chat_store[session_id] = ChatMessageHistory()
    return chat_store[session_id]

def update_long_term_memory(session_id: str, input: str, output: str):
    if session_id not in long_term_memory:
        long_term_memory[session_id] = []
    if len(input) > 20:  # Simple logic: store inputs longer than 20 characters
        long_term_memory[session_id].append(f"User said: {input}")
    if len(long_term_memory[session_id]) > 5:  # Keep only last 5 memories
        long_term_memory[session_id] = long_term_memory[session_id][-5:]

def get_long_term_memory(session_id: str):
    return ". ".join(long_term_memory.get(session_id, []))

In [3]:
# Prompt Template
# We'll create a prompt template that includes both short-term and long-term memory.

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the information from long-term memory if relevant."),
    ("system", "Long-term memory: {long_term_memory}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [4]:
# Conversational Chain
# Now, we'll set up the conversational chain with message history.

chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [5]:
# Chat Function
# We'll create a function to handle chat interactions, including updating long-term memory.

def chat(input_text: str, session_id: str):
    long_term_mem = get_long_term_memory(session_id)
    response = chain_with_history.invoke(
        {"input": input_text, "long_term_memory": long_term_mem},
        config={"configurable": {"session_id": session_id}}
    )
    update_long_term_memory(session_id, input_text, response.content)
    return response.content

In [6]:
# Example Usage
# Let's test our memory-enhanced conversational agent with a series of interactions.

session_id = "user_123"

print("AI:", chat("Hello! My name is Alice.", session_id))
print("AI:", chat("What's the weather like today?", session_id))
print("AI:", chat("I love sunny days.", session_id))
print("AI:", chat("Do you remember my name?", session_id))

AI: Hello Alice, it's nice to meet you! I'm glad to be able to assist you today.
AI: I do not have real-time access to the internet, including weather information. To find out the weather, please check a reliable weather app or website.
AI: Sunny days are great! What do you like to do on sunny days, Alice?
AI: Yes, I remember your name is Alice.


In [7]:
# Review Memory
# Let's review the conversation history and long-term memory.

print("Conversation History:")
for message in chat_store[session_id].messages:
    print(f"{message.type}: {message.content}")

print("\nLong-term Memory:")
print(get_long_term_memory(session_id))

Conversation History:
human: Hello! My name is Alice.
ai: Hello Alice, it's nice to meet you! I'm glad to be able to assist you today.
human: What's the weather like today?
ai: I do not have real-time access to the internet, including weather information. To find out the weather, please check a reliable weather app or website.
human: I love sunny days.
ai: Sunny days are great! What do you like to do on sunny days, Alice?
human: Do you remember my name?
ai: Yes, I remember your name is Alice.

Long-term Memory:
User said: Hello! My name is Alice.. User said: What's the weather like today?. User said: Do you remember my name?
